In [1]:
from treys import Deck,Card,Evaluator
import keras
import pandas as pd
import numpy as np
from keras.models import load_model

/home/csci4850/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
#dict for conversion from treys to format for NNs
trans = {'Ah': [1,1], '2h': [1,2], '3h': [1,3], '4h': [1,4],'5h': [1,5], '6h': [1,6], '7h': [1,7], '8h': [1,8], '9h': [1,9], 'Th': [1,10],'Jh': [1,11], 'Qh': [1,12], 'Kh': [1,13],
        'As': [2,1], '2s': [2,2], '3s': [2,3], '4s': [2,4],'5s': [2,5], '6s': [2,6], '7s': [2,7], '8s': [2,8], '9s': [2,9], 'Ts': [2,10],'Js': [2,11], 'Qs': [2,12], 'Ks': [2,13],
        'Ad': [3,1], '2d': [3,2], '3d': [3,3], '4d': [3,4],'5d': [3,5], '6d': [3,6], '7d': [3,7], '8d': [3,8], '9d': [3,9], 'Td': [3,10],'Jd': [3,11], 'Qd': [3,12], 'Kd': [3,13],
        'Ac': [4,1], '2c': [4,2], '3c': [4,3], '4c': [4,4],'5c': [4,5], '6c': [4,6], '7c': [4,7], '8c': [4,8], '9c': [4,9], 'Tc': [4,10],'Jc': [4,11], 'Qc': [4,12], 'Kc': [4,13]
        } 

In [36]:
#will probably have to replicate all of this code for the
#2nd NN, so just a copy/paste with var name changes inside this loop

NN1_wins=0
dumb_luck_wins=0
num_games=0

for y in range(0,100): #loop game
    num_games+=1
    
    deck=Deck() #initialize vars 
    evaluator=Evaluator()
    
    player1_hand=deck.draw(5) #draw initial hands
    nn_hand=deck.draw(5)

    print ("Player1 hand: ")# prints initial hands
    Card.print_pretty_cards(player1_hand)
    print ("NN hand: ")
    Card.print_pretty_cards(nn_hand)
    
    
    
    
    #determine what the NN thinks of the hand
    hand_for_nn=[]
    for i in range(0,5):
        hand_for_nn=(hand_for_nn+trans[Card.int_to_str(nn_hand[i])])

    
    
    model = load_model('ffbnn.h5') #ffbnn.h5 is feed forward NN

    handarray = np.array([hand_for_nn]) #convert hand to np array for input

    preds = model.predict(handarray) #calculates probabilities of each class from NN

    label=np.argmax(preds)

    print("Label", label)
    
    
    #determine if NN was correct
    player1_score=evaluator._five(player1_hand)
    ordinal=evaluator._five(nn_hand)

    #convert ordinal rank from treys to NN value
    end=0

    if(ordinal > 6185): #high card("not a hand")
        end = 0
    elif(ordinal > 3325): #one pair
        end = 1
    elif(ordinal > 2467): #two pair
        end = 2
    elif(ordinal > 1609): #three of a kind
        end = 3
    elif(ordinal > 1599): #straight
        end = 4
    elif(ordinal > 322): #flush
        end = 5
    elif(ordinal > 166): #full house
        end = 6
    elif(ordinal > 10): #four of a kind
        end = 7
    elif(ordinal > 1): #straight flush
        end = 8
    elif(ordinal == 1): #royal flush
        end = 9
    
    
    if (label==end):
        print("nn classified correctly")
        #do normal algos
    
    else:
        print("nn is stupid")
        label=0 #just simple skip rest of logical code, kind of dirty but its easier rn
    
    
    
    
    #decision making
    #if nothing in hand
    if (label==0):
        print("Label 0 was chosen, remove after testing")
        new_hand=deck.draw(5)#draw 5 new cards
        print ("NN hand: ")
        Card.print_pretty_cards(new_hand)
        nn_hand=new_hand
    
    #if one pair
    if (label==1):
        nn_hand.sort() #sort arr by val
        print("Label 1 was chosen: remove after testing")
        #need to determine what cards are the pair.
        #i think a pair just has the same value, dif suit so should be
        #checking every even ele if beginning count at 0
    
        cards_to_keep=[0,0] #array where will store the cards from hand to keep, init as strings for testing
        new_hand=[0,0] #actual arr where cards will be stored as their int encodings
        count=2
        for i in range(0,4):
            current=Card.int_to_str(nn_hand[i]) #get card as str
            current_num=nn_hand[i]
            for x in range(0,5):
                if (x!=i): #if we're not on the card we are checking for a match for
                    check=Card.int_to_str(nn_hand[x]) #get card to check against current
                    check_num=nn_hand[x]
                    if(check[0]==current[0]): #if the cards have the same value aka are a pair
                        cards_to_keep[0]=check
                        cards_to_keep[1]=current
                        
                        new_hand[0]=check_num 
                        new_hand[1]=current_num
                    
                    
                    
        draw_3=deck.draw(3)#draw 3 new cards
        new_hand=new_hand+draw_3 #add the 3 new cards to the hand, keeping the pairs
                    
        Card.print_pretty_cards(new_hand)
        nn_hand=new_hand
        #for y in range(0,5):
         #   print(Card.int_to_str(new_hand[y]))
                
            
    if (label==2): #two pairs
        print("Label 2 was chosen, remove post testing")
    
   
    
        nn_hand.sort() #sort arr by val
    
        cards_to_keep=[0,0,0,0] #array where will store the cards from hand to keep, init as strings for testing
        new_hand=[0,0,0,0] #actual arr where cards will be stored as their int encodings
        count=0
        has_1_pair=0
        for i in range(0,4):
            current=Card.int_to_str(nn_hand[i]) #get card as str
            current_num=nn_hand[i]
            if(i==4):
                break
            for x in range(0,5):
                if (x!=i): #if we're not on the card we are checking for a match for
                    check=Card.int_to_str(nn_hand[x]) #get card to check against current
                    check_num=nn_hand[x]
                    if(check[0]==current[0] and (check_num!=new_hand[0] and check_num!=new_hand[1])): #if the cards have the same value aka are a pair 
                        for y in range(0,4):
                            if (check_num not in new_hand): #man python is nice
                                cards_to_keep[count]=check
                                new_hand[count]=check_num
                                count+=1
                                cards_to_keep[count]=current
                                new_hand[count]=current_num
                                count+=1
                                #print(len(new_hand))
                    
                   
        draw_1=deck.draw(1)#draw 1 new card
        drawn=[draw_1]
        new_hand=new_hand+drawn #add the new card to the hand, keeping the pairs
                
        #Card.print_pretty_cards(new_hand)
        
        #DONT DELETE, might need
       # for y in range(0,5):
          #  print(Card.int_to_str(new_hand[y]))
        Card.print_pretty_cards(new_hand)
        
        nn_hand=new_hand
    
    
    
       
    
       
    
    
    if (label==3): #three of a kind
        nn_hand.sort() #sort arr by val
        print("Label 3 was chosen, remove post testing")
        cards_to_keep=[0,0,0] #array where will store the cards from hand to keep, init as strings for testing
        new_hand=[0,0,0] #actual arr where cards will be stored as their int encodings
        count=0
        for i in range(0,4):
            current=Card.int_to_str(nn_hand[i]) #get card as str
            current_num=nn_hand[i]
            for x in range(0,4):
                if (x!=i): #if we're not on the card we are checking for a match for
                    check=Card.int_to_str(nn_hand[x]) #get card to check against current
                    check_num=nn_hand[x]
                    if(check[0]==current[0]): #if the cards have the same value aka are a pair
                        cards_to_keep[0]=check
                        cards_to_keep[1]=current
                    
                        new_hand[0]=check_num 
                        new_hand[1]=current_num
                   
                    
                        #find third card
                        for y in range(0,5):
                            check2=Card.int_to_str(nn_hand[y])
                            if(check2[0]==current[0] and (check2[1]!=check[1] and check2[1]!=current[1])): #if card is same val as other 2 and not already in the list to keep #can also use not in list
                                store=nn_hand[y]
                            
                                new_hand[2]=new_hand[1]
                                new_hand[1]=store
                                cards_to_keep[2]=check2
                    
                    
                   
        draw_2=deck.draw(2)#draw 2 new cards
        new_hand=new_hand+draw_2 #add the 2 new cards to the hand, keeping the pairs
        Card.print_pretty_cards(new_hand)
        nn_hand=new_hand
   
    

    if(label>3):
        print("I have a really good hand.")
        
    
    
    
    #using rankings of evaluator from Treys to determine winner

    #get scores
    player1_score=evaluator._five(player1_hand)
    nn_score=evaluator._five(new_hand)

    if (nn_score<player1_score):
        print("NN wins!")
        NN1_wins+=1
    else:
        print("Dumb luck wins!")
        dumb_luck_wins+=1
    
    print ("Player1 hand: ")# prints final hands 
    Card.print_pretty_cards(player1_hand)

    print ("NN hand: ")
    Card.print_pretty_cards(new_hand)
        
        
    
    
    
        
                
    
    
    
    

Player1 hand: 
 [2♣],[8♣],[8♦],[K♣],[2♥] 
NN hand: 
 [A♦],[7♠],[T♥],[3♣],[A♣] 
Label 1
nn classified correctly
Label 1 was chosen: remove after testing
 [A♣],[A♦],[6♣],[Q♣],[5♠] 
Dumb luck wins!
Player1 hand: 
 [2♣],[8♣],[8♦],[K♣],[2♥] 
NN hand: 
 [A♣],[A♦],[6♣],[Q♣],[5♠] 
Player1 hand: 
 [Q♣],[9♣],[4♣],[K♠],[5♠] 
NN hand: 
 [6♦],[9♥],[5♦],[2♣],[T♣] 
Label 0
nn classified correctly
Label 0 was chosen, remove after testing
NN hand: 
 [5♣],[6♠],[7♦],[3♠],[Q♦] 
Dumb luck wins!
Player1 hand: 
 [Q♣],[9♣],[4♣],[K♠],[5♠] 
NN hand: 
 [5♣],[6♠],[7♦],[3♠],[Q♦] 
Player1 hand: 
 [9♥],[8♦],[5♣],[5♥],[J♠] 
NN hand: 
 [7♥],[9♦],[9♣],[K♠],[J♣] 
Label 1
nn classified correctly
Label 1 was chosen: remove after testing
 [9♦],[9♣],[6♣],[A♥],[Q♥] 
NN wins!
Player1 hand: 
 [9♥],[8♦],[5♣],[5♥],[J♠] 
NN hand: 
 [9♦],[9♣],[6♣],[A♥],[Q♥] 
Player1 hand: 
 [T♠],[K♠],[K♥],[4♠],[6♥] 
NN hand: 
 [5♦],[Q♠],[8♥],[8♣],[6♣] 
Label 1
nn classified correctly
Label 1 was chosen: remove after testing
 [8♥],[8♣],[8♦],[4♣],[9

In [37]:
print("NN1 wins:", '{:.1%}'.format(NN1_wins/num_games), "of the time.")
print("Dumb luck wins:", '{:.1%}'.format(dumb_luck_wins/num_games), "of the time.")

NN1 wins: 66.0% of the time.
Dumb luck wins: 34.0% of the time.
